In [158]:
import json
import pandas as pd

def transform_test_set(path_to_json='test.json'):
    data = json.load(open(path_to_json))
    values = []
    for i in range(len(data)):
        for w in data[i]:
            values.append({'sentense_id':i, "word": w[0], 'value': w[1]})            
    
    df = pd.DataFrame(values)
    df['id']=df.index.values
    df['value']=True #predict all words as the end of the sentense
    df[['id','value']].to_csv('sample.csv', index=False, encoding='utf8')
    
transform_test_set()

In [159]:
data = json.load(open('test.json'))

In [161]:
rows=[]
for row in data:
    sents = []
    sent =[]
    for word in row:
        sent.append(word[0])
        if word[1]==True:
            sents.append(' '.join(sent))
            sent=[]
    sents.append(' '.join(sent))
    rows.append(sents)

# How the run on test data looks like:

In [164]:
for row in rows:
    print(row)
    print('----------------------------------')

['I think the magnitude of a benefit and error rates that were chosen were reasonable', 'They were standard from our learning .']
----------------------------------
['Economists on both the left and right broadly agree that the need for stimulative government spending is necessary to prevent a further collapse of the global economic system - just as the New Deal and the deficit spending of World War II restored the health of the global economy in the last century .']
----------------------------------
['I may be there some time', 'it looks like a nice place to chill out and walk around for a few hours .']
----------------------------------
['Due to this , the new art was based on real events occurring in Europe and represented in decadent culture with a moral character', 'The art incorporated both political and social roles as the movement evolved .']
----------------------------------
['It is not necessary to make the conflict because of various teaching styles', 'Teachers could seek 

In [ ]:
preprocessing and data collectiong
    Need to find some short conversation, probably twitter or sms, something like that
    check is tokenization work in the same way as in the test

Features:
    
    lematized end word and next word of splitting
    is the first character uppercase
    pos of the word
    dependensy path 
    length of the first word
    is it popular 2gram 0-1 of end and next word start
    

In [1]:
# source https://drive.google.com/open?id=13jXBNgZhLKg95JjyArKm-muoXSoP33TG
import pandas as pd
tweets= pd.read_csv('tweets.csv', encoding="cp1252")
tweets.columns= ['index','q','d','n','a', 'text']
tweets=tweets[['text']]

In [2]:
len(tweets)

1599998

In [3]:
tweets[:5].values

array([['@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'],
       ['my whole body feels itchy and like its on fire '],
       ["@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "],
       ['@Kwesidei not the whole crew '],
       ['Need a hug ']], dtype=object)

# preprocessing replace all the @name on name or hi , shi ,it, name

In [6]:
import random
import re

def replace_twitter_name_with_normal_one(text):
    names_list=['He','he','she', 'She', 'It', 'we', 'We', 'they', 'i','I', 'you', 'u', 'You', 'Tom', 'Bill' , 'Mazar', 'Mike', 'nazar']
    index = random.randint(0, len(names_list)-1)
    new_name=names_list[index]
    return re.sub(r'@[a-z0-9A-Z]*', new_name, text.strip())

replace_twitter_name_with_normal_one('@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds')

'He I dived many times for the ball. Managed to save 50%  The rest go out of bounds'

In [7]:
tweets['text'] = tweets['text'].apply(replace_twitter_name_with_normal_one)

In [8]:
tweets['text'][:5].values

array(['they I dived many times for the ball. Managed to save 50%  The rest go out of bounds',
       'my whole body feels itchy and like its on fire',
       "Mike no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",
       'She not the whole crew', 'Need a hug'], dtype=object)

# Prepare training data

In [9]:
from nltk import tokenize
tokenize.sent_tokenize('Tom I dived many times for the ball. Managed to save 50%  The rest go out of bounds')

['Tom I dived many times for the ball.',
 'Managed to save 50%  The rest go out of bounds']

In [10]:
#get all last characters of the sentense
last_chars=set()
for text in tweets['text'].values:
    sentenses = tokenize.sent_tokenize(text)
    for sentense in sentenses:
        last_chars.add(sentense[-1])
        
''.join(list(last_chars))

"lob:»Ÿ“%¸¼µ}BQ*X›HOjeq‘wm¯0/¢„VR˜.'(¨uFž¤°N‹¿UÂaDh½s\xad8k|¦{fˆ–1†^5=]ƒg—·\\3[z?¬!´Š³…W©t™i±$62TP¾GE¥”#¹c`S®C€Y_A)Ã4Ž‡9š’œZŒ£nI§¡•‰M,;pKxrL~‚vy«-²¶Jº7+dª"

In [46]:
import numpy as np
import tqdm

def transform_tweets_to_train_format(texts):
    train_data=[]
    for text_index in tqdm.tqdm(range(len(texts))):
        labeled_words=[]
        sentenses = tokenize.sent_tokenize(texts[text_index])
        for sentens_index, sentense in enumerate(sentenses): 
            if len(sentense)<2:
                continue
            words = tokenize.word_tokenize(sentense if sentense[-1] not in "?!;.¸‚" else sentense[:-1])
            values = [False]*len(words)
            try:
                values[-1]=len(sentenses)>(sentens_index+1)
            except:
                print('Data:' ,texts[text_index])
                raise ValueError('A very specific bad thing happened.')
            labeled_words.append(list(zip(words,values)))
            
        if len(labeled_words)>0:
            train_data.append(np.concatenate(labeled_words))
    return train_data

In [ ]:
#lower case of next sentense posibli will be needed

In [47]:
transform_tweets_to_train_format(['Tom I dived many times for the ball. Managed to save 50%  The rest go out of bounds'])


100%|██████████| 1/1 [00:00<00:00, 666.82it/s]

[array([['Tom', 'False'],
        ['I', 'False'],
        ['dived', 'False'],
        ['many', 'False'],
        ['times', 'False'],
        ['for', 'False'],
        ['the', 'False'],
        ['ball', 'True'],
        ['Managed', 'False'],
        ['to', 'False'],
        ['save', 'False'],
        ['50', 'False'],
        ['%', 'False'],
        ['The', 'False'],
        ['rest', 'False'],
        ['go', 'False'],
        ['out', 'False'],
        ['of', 'False'],
        ['bounds', 'False']], dtype='<U7')]

In [48]:
train = transform_tweets_to_train_format(tweets['text'].values)


  0%|          | 6767/1599998 [00:01<07:20, 3614.18it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "Z:\anaconda\envs\py_36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "Z:\anaconda\envs\py_36\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "Z:\anaconda\envs\py_36\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 1599998/1599998 [07:03<00:00, 3781.42it/s]


In [57]:
train[:2]

[array([['they', 'False'],
        ['I', 'False'],
        ['dived', 'False'],
        ['many', 'False'],
        ['times', 'False'],
        ['for', 'False'],
        ['the', 'False'],
        ['ball', 'True'],
        ['Managed', 'False'],
        ['to', 'False'],
        ['save', 'False'],
        ['50', 'False'],
        ['%', 'False'],
        ['The', 'False'],
        ['rest', 'False'],
        ['go', 'False'],
        ['out', 'False'],
        ['of', 'False'],
        ['bounds', 'False']], dtype='<U7'), array([['my', 'False'],
        ['whole', 'False'],
        ['body', 'False'],
        ['feels', 'False'],
        ['itchy', 'False'],
        ['and', 'False'],
        ['like', 'False'],
        ['its', 'False'],
        ['on', 'False'],
        ['fire', 'False']], dtype='<U5')]

In [58]:
train.save('train.txt')

AttributeError: 'list' object has no attribute 'save'

In [56]:
import json
with open('train.json', 'w') as outfile:
    json.dump(train, outfile)

TypeError: Object of type 'ndarray' is not JSON serializable